In [ ]:
!pip install mlrun==0.6.0-rc3


In [9]:
from mlrun import new_function
from mlrun.platforms.iguazio import mount_v3io, mount_v3iod
import os
image_name = 'quay.io/iguazio/shell:' + os.environ.get("IGZ_VERSION")
spark_func = new_function(name='my-spark' , command='/v3io/users/avia/parquez/functions/kv_to_parquet.py', kind='job', mode='pass')
spark_func.apply(mount_v3io(name="v3io-fuse", remote="/", mount_path="/v3io"))
spark_func.apply(mount_v3iod(namespace="default-tenant", v3io_config_configmap="spark-operator-v3io-config"))
spark_func.spec.build.commands = ['pip install python-dateutil']
spark_func.spec.build.base_image = image_name
spark_func.deploy(with_mlrun=False)
spark_func.run(artifact_path="/User/artifacts")

> 2020-12-07 14:22:48,536 [info] starting remote build, image: .mlrun/func-default-my-spark-latest
INFO[0000] Resolved base name quay.io/iguazio/shell:3.0_b5790_20201130062230 to quay.io/iguazio/shell:3.0_b5790_20201130062230 
INFO[0000] Resolved base name quay.io/iguazio/shell:3.0_b5790_20201130062230 to quay.io/iguazio/shell:3.0_b5790_20201130062230 
INFO[0000] Retrieving image manifest quay.io/iguazio/shell:3.0_b5790_20201130062230 
INFO[0001] Retrieving image manifest quay.io/iguazio/shell:3.0_b5790_20201130062230 
INFO[0002] Built cross stage deps: map[]                
INFO[0002] Retrieving image manifest quay.io/iguazio/shell:3.0_b5790_20201130062230 
INFO[0002] Retrieving image manifest quay.io/iguazio/shell:3.0_b5790_20201130062230 
INFO[0003] Unpacking rootfs as cmd RUN pip install python-dateutil requires it. 
INFO[0073] Taking snapshot of full filesystem...        
INFO[0075] Resolving paths                              
INFO[0079] RUN pip install python-dateutil           

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...9beca47e,0,Dec 07 14:24:21,completed,my-spark,v3io_user=aviakind=jobowner=avia,,,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 6db0abd57b1944aaab385f9c9beca47e --project default , !mlrun logs 6db0abd57b1944aaab385f9c9beca47e --project default
> 2020-12-07 14:24:34,246 [info] run executed, status=completed
